# Baseline

always predict the majority class

In [11]:
import pandas as pd

# Load the pre-split data
X_train = pd.read_csv("../2_data/X_train.csv")
X_test = pd.read_csv("../2_data/X_test.csv")
y_train = pd.read_csv("../2_data/y_train.csv").squeeze()  # Use squeeze to convert DataFrame to Series
y_test = pd.read_csv("../2_data/y_test.csv").squeeze()

In [14]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate_baseline(y_test, predictions):
    base_accuracy = accuracy_score(y_test, predictions)
    base_precision = precision_score(y_test, predictions, pos_label=majority_class)
    base_recall = recall_score(y_test, predictions, pos_label=majority_class)
    base_f1_weighted = f1_score(y_test, predictions, average='weighted')
    print(f"Accuracy: {base_accuracy:.6f}")
    print(f"Precision: {base_precision:.6f}")
    print(f"Recall: {base_recall:.6f}")
    print(f"F1-score: {base_f1_weighted:.6f}")
    return base_f1_weighted

# Compute predictions (always predict majority class)
majority_class = y_train.mode()[0]
predictions = [majority_class] * len(y_test)

# Evaluate baseline model
print("Baseline performance predicting always the majority class")
majority_baseline_f1 = evaluate_baseline(y_test, predictions)

Baseline performance predicting always the majority class
Accuracy: 0.734564
Precision: 0.734564
Recall: 1.000000
F1-score: 0.622155
